In [1]:
#Part1 import
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
# import torchvision.datasets as datasets
# from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [2]:
#Part2 Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 0.0002
batch_size = 128
num_epochs = 50
latent_size = 100#
image_size = 28 * 28 * 1  # 784
# z_dim = 64

In [3]:
#Part3 data
transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

dataset = datasets.MNIST(root="dataset/", transform=transforms, train = True, download=False)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
#Part4 class
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.label_emb = nn.Embedding(10, 10)
        self.disc = nn.Sequential(
            nn.Linear(latent_size + 10, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
        emb = self.label_emb(labels)
        x = torch.cat([x, emb], 1)
        x = x.view(x.size(0), -1)
        return self.disc(x)

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.label_emb = nn.Embedding(10, 10)
        self.gen = nn.Sequential(
            nn.Linear(latent_size + 10, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, image_size),
            nn.Tanh()
        )

    def forward(self, x, labels):
        emb = self.label_emb(labels)
        x = torch.cat([x, emb], 1)
        return self.gen(x)

In [5]:
#Part5 before train
disc = Discriminator().to(device)
gen = Generator().to(device)

# fixed_noise = torch.randn((batch_size, z_dim)).to(device)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()

In [ ]:
#Part6 Train
for epoch in range(num_epochs):
    for batch_idx, (images, labels) in enumerate(loader):
        batch_size = images.shape[0]
        real_images = images.view(batch_size, -1).to(device)
        real_lables = labels.to(device)

        disc_real = disc(real_images, real_lables)
        lossD_real = criterion(disc_real, torch.ones(batch_size, 1).to(device))

        noise = torch.randn(batch_size, latent_size).to(device)
        fake_images = gen(noise, labels.to(device))

        


